In [16]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.regression.mixed_linear_model import MixedLM
from scipy.stats import t as t_dist
from statsmodels.formula.api import mixedlm

In [17]:
density_mergefee.columns

Index(['City', 'County', 'Count', 'RetailerYear', 'RetailerCount_density',
       'PopulationYear', 'Black_or_African_American',
       'American_Indian_and_Alaska_Native', 'Asian',
       'Native_Hawaiian_and_Other_Pacific_Islander', 'Some_Other_Race',
       'Two_or_More_Races', 'Total_RacialMinority',
       'Percent_below_poverty_level', 'Highschool_higher_1824',
       'Bachelor_higher_1824', 'Highschool_higher_25', 'Bachelor_higher_25',
       'smokefree', 'citylicense', 'pharmacyban', 'emerging_license',
       'retailrestrict', 'flavored', 'EnactmentDate', 'AnnualFee',
       'Application_Initial_Fee', 'year', 'EnactmentYear'],
      dtype='object')

In [18]:
# Set path
path = 'D:\\Purdue\\research\\lmer\\'

# Read data
density_mergefee = pd.read_csv(path + "density_mergefee_lmer_mile.csv")

density_mergefee.columns = density_mergefee.columns.str.replace('.', '_')

# Summary of selected columns
print(density_mergefee.iloc[:, [5, 13, 14, 16, 18, 20, 21, 28]].describe())

# Remove rows with missing values
my_data = density_mergefee.iloc[:, [13, 14, 16, 18, 20, 21, 28]].dropna()

# Function to create coefficient table
def coef_table(did_main):
    # Extract fixed effects coefficients and standard errors
    coef = did_main.params
    std_err = did_main.bse
    df = len(density_mergefee) - len(coef) - 1
    tval = coef / std_err
    pval = 2 * (1 - t_dist.cdf(abs(tval), df=df))
    stars = ['***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else '' for p in pval]
    alpha = 0.05
    zval = t_dist.ppf(1 - alpha / 2, df=df)
    ci_lo = coef - zval * std_err
    ci_hi = coef + zval * std_err
    ci_str = [f"({lo:.2f}-{hi:.2f}){star}" for lo, hi, star in zip(ci_lo, ci_hi, stars)]
    coef_table = pd.DataFrame({'Estimate': coef, 'CI and Significance': ci_str})
    return coef_table

       PopulationYear  Percent_below_poverty_level  Highschool_higher_1824  \
count     4894.000000                  4894.000000             4894.000000   
mean      2012.686964                     0.150481                0.849673   
std          4.433270                     0.084481                0.100231   
min       2010.000000                     0.015000                0.000000   
25%       2010.000000                     0.083000                0.810000   
50%       2010.000000                     0.137000                0.870000   
75%       2020.000000                     0.200750                0.910000   
max       2020.000000                     0.495000                1.000000   

       Highschool_higher_25    smokefree  pharmacyban  emerging_license  \
count           4894.000000  4434.000000  4434.000000       3082.000000   
mean               0.814706     1.285295     0.049617          0.277417   
std                0.137729     1.448422     0.217176          0.447797 

C:\Users\Steven\AppData\Local\Temp\ipykernel_11504\4192539685.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  density_mergefee.columns = density_mergefee.columns.str.replace('.', '_')


In [36]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm

# Ensure 'citylicense' is categorical if it's not numeric
density_mergefee['citylicense'] = pd.Categorical(density_mergefee['citylicense'])
density_mergefee['City'] = pd.Categorical(density_mergefee['City'])

# Ensure no NA values that could be causing issues
density_mergefee.dropna(inplace=True)

# Main model using from_formula method
did_main_formula = 'RetailerCount_density ~ year + citylicense + pharmacyban + Total_RacialMinority + Bachelor_higher_1824 + Bachelor_higher_25 + Percent_below_poverty_level + year:citylicense + year:pharmacyban'
did_main = MixedLM.from_formula(did_main_formula, groups=density_mergefee['City'], data=density_mergefee)
did_main_result = did_main.fit()
print(did_main_result.summary())

# Coefficient table
coef_table_main = pd.DataFrame(did_main_result.params, columns=['Estimate'])
coef_table_main.index.name = 'Variable'
coef_table_main.to_csv("main__.csv")  # Ensure path is correct or defined

# Perform the Adjusted Wald test
adjusted_wald_test = did_main_result.wald_test_terms().summary_frame()
print(adjusted_wald_test)

# # Likelihood ratio test
# X = sm.add_constant(density_mergefee[['year']], has_constant='add') 
# null_model = sm.OLS(density_mergefee['RetailerCount_density'], X['const']).fit()
# lrtest = sm.stats.anova_lm(null_model, did_main_result, typ=1)
# print(lrtest)

                  Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   RetailerCount_density
No. Observations:    174       Method:               REML                 
No. Groups:          26        Scale:                0.8549               
Min. group size:     1         Log-Likelihood:       -289.0070            
Max. group size:     12        Converged:            Yes                  
Mean group size:     6.7                                                  
--------------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z|  [0.025   0.975]
--------------------------------------------------------------------------
Intercept                    11.839    2.531  4.677 0.000    6.877  16.800
citylicense[T.strong]         1.866    0.926  2.015 0.044    0.051   3.681
citylicense[T.weak]           1.881    0.829  2.270 0.023    0.257   3.505
year                          0.261    0.615

C:\Users\Steven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


In [37]:
# Ethnicity model
did_ethnicity_formula = 'RetailerCount_density ~ year + citylicense + pharmacyban + Total_RacialMinority + Bachelor_higher_1824 + Bachelor_higher_25 + Percent_below_poverty_level + year:citylicense + year:pharmacyban + year:Total_RacialMinority + citylicense:Total_RacialMinority + pharmacyban:Total_RacialMinority + year:citylicense:Total_RacialMinority'
did_ethnicity = mixedlm(did_ethnicity_formula, data=density_mergefee, groups=density_mergefee['City'])
did_ethnicity_result = did_ethnicity.fit()
print(did_ethnicity_result.summary())

# Coefficient table
coef_table_ethnicity = pd.DataFrame(did_ethnicity_result.params, columns=['Estimate'])
coef_table_ethnicity.index.name = 'Variable'
coef_table_ethnicity.to_csv(path + "ethnicity__.csv")

# Perform the Adjusted Wald test
adjusted_wald_test_ethnicity = did_ethnicity_result.wald_test_terms().summary_frame()
print(adjusted_wald_test_ethnicity)

                              Mixed Linear Model Regression Results
Model:                       MixedLM           Dependent Variable:           RetailerCount_density
No. Observations:            174               Method:                       REML                 
No. Groups:                  26                Scale:                        0.7940               
Min. group size:             1                 Log-Likelihood:               -240.6999            
Max. group size:             12                Converged:                    Yes                  
Mean group size:             6.7                                                                  
--------------------------------------------------------------------------------------------------
                                                  Coef.   Std.Err.   z    P>|z|   [0.025   0.975] 
--------------------------------------------------------------------------------------------------
Intercept                                

C:\Users\Steven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


In [38]:

# Poverty model
# did_poverty_formula = 'RetailerCount_density ~ year + citylicense + pharmacyban + Total_RacialMinority + Bachelor_higher_1824 + Bachelor_higher_25 + Percent_below_poverty_level + year:citylicense + year:pharmacyban + year:Percent_below_poverty_level + citylicense:Percent_below_poverty_level + pharmacyban:Percent_below_poverty_level + year:Percent_below_poverty_level:citylicense + year:Percent_below_poverty_level:pharmacyban'
# did_poverty = mixedlm(did_poverty_formula, data=density_mergefee, groups=density_mergefee['City'])
# did_poverty_result = did_poverty.fit()
# print(did_poverty_result.summary())

# Check correlations among predictors
predictors = ['year', 'pharmacyban', 'Total_RacialMinority', 'Bachelor_higher_1824', 'Bachelor_higher_25', 'Percent_below_poverty_level']
corr_matrix = density_mergefee[predictors].corr()
print(corr_matrix)

# Calculate VIF for each predictor
from statsmodels.stats.outliers_influence import variance_inflation_factor
X = pd.get_dummies(density_mergefee[predictors], drop_first=True)
X = sm.add_constant(X)
vif = pd.DataFrame([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns, columns=['VIF'])
print(vif)

# Simplify the model by removing some interactions
simplified_formula = 'RetailerCount_density ~ year + citylicense + pharmacyban + Total_RacialMinority + Bachelor_higher_1824 + Bachelor_higher_25 + Percent_below_poverty_level + year:citylicense + year:pharmacyban'
simplified_model = mixedlm(simplified_formula, data=density_mergefee, groups=density_mergefee['City'])
simplified_result = simplified_model.fit()
print(simplified_result.summary())

# Coefficient table
coef_table_poverty = pd.DataFrame(did_poverty_result.params, columns=['Estimate'])
coef_table_poverty.index.name = 'Variable'
coef_table_poverty.to_csv(path + "poverty__.csv")

# Perform the Adjusted Wald test
adjusted_wald_test_poverty = did_poverty_result.wald_test_terms().summary_frame()
print(adjusted_wald_test_poverty)

# Education model
did_education_formula = 'RetailerCount_density ~ year + citylicense + pharmacyban + Total_RacialMinority + Bachelor_higher_1824 + Bachelor_higher_25 + Percent_below_poverty_level + (1 | City) + year:citylicense + year:pharmacyban + year:Bachelor_higher_1824 + year:Bachelor_higher_25 + citylicense:Bachelor_higher_1824 + pharmacyban:Bachelor_higher_1824 + citylicense:Bachelor_higher_25 + pharmacyban:Bachelor_higher_25 + year:Bachelor_higher_1824:citylicense + year:Bachelor_higher_1824:pharmacyban + year:Bachelor_higher_25:citylicense + year:Bachelor_higher_25:pharmacyban'
did_education = mixedlm(did_education_formula, data=density_mergefee, groups=density_mergefee['City'])
did_education_result = did_education.fit()
print(did_education_result.summary())

# Coefficient table
coef_table_education = pd.DataFrame(did_education_result.params, columns=['Estimate'])
coef_table_education.index.name = 'Variable'
coef_table_education.to_csv(path + "education__.csv")

# Perform the Adjusted Wald test
adjusted_wald_test_education = did_education_result.wald_test_terms().summary_frame()
print(adjusted_wald_test_education)

                                 year  pharmacyban  Total_RacialMinority  \
year                         1.000000     0.085233              0.047363   
pharmacyban                  0.085233     1.000000              0.080260   
Total_RacialMinority         0.047363     0.080260              1.000000   
Bachelor_higher_1824         0.125345     0.277478             -0.323846   
Bachelor_higher_25           0.106328     0.198585             -0.366319   
Percent_below_poverty_level -0.047473    -0.087119              0.344498   

                             Bachelor_higher_1824  Bachelor_higher_25  \
year                                     0.125345            0.106328   
pharmacyban                              0.277478            0.198585   
Total_RacialMinority                    -0.323846           -0.366319   
Bachelor_higher_1824                     1.000000            0.817205   
Bachelor_higher_25                       0.817205            1.000000   
Percent_below_poverty_level  

NameError: name 'did_poverty_result' is not defined